## Scrapping urbania

The goal of this notebook is to scrape data from https://urbania.pe/ and store it on a mongo databse

### Libraries import

In [144]:
import math
import numpy as np
import pymongo
import re 
import urllib.request as urllib2
import datetime

from time import sleep
from bs4 import BeautifulSoup

from constants import *

In [103]:
# bsae urls to use
base_url = 'https://urbania.pe{}'
apartments_url = 'https://urbania.pe/buscar/venta-de-departamentos'


### Define database connections

In [145]:
uri = "mongodb://{}:{}@{}/{}".format(username, password, hostlist, database)
client = pymongo.MongoClient(uri)
db = client['urbania']

In [146]:
list(db['admin'].find())

[{'_id': ObjectId('5deb3aba3dadd940d2148a36'), 'name': 'gonzalo'},
 {'_id': ObjectId('5dec891299df02824e747081'), 'name': 'jay'}]

In [77]:
apartmentsdb = db['apartments']

### Here we put the logic to get the data

In [53]:
def getSoupObj(url):
    sleep(1.5)
    try:
        r = urllib2.urlopen(url)
    except UnicodeEncodeError as e:
        return None
    html = r.read()
    htmltext = html.decode('utf-8')
    return BeautifulSoup(htmltext, 'lxml')

In [142]:
tihs = 10000
count = 1
while count < 5:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(apartments_url, headers=hdr)
    apartmentsBsObj = getSoupObj(req)
    
    objs = apartmentsBsObj.findAll('article', {'class': 'b-card-item js-card-item b-card-item--proyecto-destacado'})
#     objsSeen = apartmentsBsObj
    print('In page {} we found {} apartments'.format(count, len(objs)))
    for obj in objs:
        
#         the objA is the object with <a> label in obj
        objA = obj.find('a')
        if objA is None: continue
            
#         objA = objA.text
        apartment = {}
        
        apartment['_id'] = obj['data-identifier']
        apartment['url'] = objA['href']
#         print(type(obj), type(objA))
#         print(obj['data-identifier'])
        get_apartment(apartment)
        print(apartment)
#         break
    count +=1
    break

In page 1 we found 15 apartments
{'_id': '4791', 'url': '/inmueble/proyecto-san-tadeo-173-lima-santiago-de-surco-gs-proyecto-inmobiliario-sac-4791', 'title': 'Proyecto San Tadeo 173', 'address': 'Calle San Tadeo #173 Santiago De Surco, Lima, Lima', 'features': '128.00 - 229.00 m2 | 2 a 3 Dorm. | 2 Baños'}
{'_id': '4174', 'url': '/inmueble/proyecto-residencial-san-martin-lima-magdalena-del-mar-wallong-peru-sac-4174', 'title': 'Proyecto Residencial San Martin', 'address': 'Jr. San Martin 514, Magdalena  Magdalena Del Mar, Lima, Lima', 'features': '70.00 - 88.11 m2 | 2 a 3 Dorm. | 2 Baños'}
{'_id': '4514', 'url': '/inmueble/proyecto-w-worldclass-ya-iniciamos-obra-lima-san-isidro-aviva-inmobiliaria--4514', 'title': 'Proyecto "W-WORLDCLASS" - YA INICIAMOS OBRA', 'address': 'Calle Los Cisnes 395  San Isidro, Lima, Lima', 'features': '72.31 - 264.01 m2 | 1 a 3 Dorm. | 1 a 4 Baños'}
{'_id': '4712', 'url': '/inmueble/proyecto-natural-lima-san-borja-aviva-inmobiliaria--4712', 'title': 'Proyecto 

In [147]:
def get_apartment(apartment):
    
    apartment_url = base_url.format(apartment['url'])
#     print(apartment_url)
    req = urllib2.Request(apartment_url)
    apartmentObj = getSoupObj(req)
#     print(apartmentObj)
    
    if apartmentObj == -1:
        return
    
    get_text(apartment, apartmentObj, 'title', 'e-title e-subtitle--with-logo', 'h1')
    get_text(apartment, apartmentObj, 'address', 'e-subtitle e-subtitle--with-logo', 'p')
    get_text(apartment, apartmentObj, 'features', 'e-features', 'p')
    get_text(apartment, apartmentObj, 'intial_price', 'e-totalPrice', 'p')
    get_text(apartment, apartmentObj, 'intial_price', 'e-totalPrice', 'p')
    
#     print(apartment)
#     print(address.text)

In [138]:
def get_text(obj, objBs, feature, tag, marker):
    new_feature = objBs.find(marker, {'class': tag})
    obj[feature] = new_feature.text